In [1]:
import os
os.chdir('..')

from puzzle_generator import *

In [2]:
width = 5
height = 5
puzzle = RectangularPuzzle(
    width=width,
    height=height
)

puzzle.solving_timeout = 300

puzzle.naming = {
    None: '   ',
    'r': ' r ',
    'p': ' p ',
    's': ' s ',
    '0': ' 0 ',
    'mine(r)': '[r]',
    'mine(p)': '[p]',
    'mine(s)': '[s]',
}

puzzle.domain_program += f"""
    sign(r;p;s).
    value(S) :- sign(S).
    value(mine(S)) :- sign(S).
    value(0).
"""

puzzle.puzzle_gen_program += """
    { puzzle(C,V) : cell(C), value(V) }.
    :- puzzle(C,mine(S)), cell(C).
    :- puzzle(C,0), cell(C).
"""
puzzle.puzzle_constraints_program += """
    #minimize { 1,puzzle(C,V) : puzzle(C,V) }.
    
    :- not solution(C,r) : cell(C).
    :- not solution(C,p) : cell(C).
    :- not solution(C,s) : cell(C).
"""

puzzle.solution_program = """
    1 { solution(C,V) : value(V) } 1 :- cell(C).
    solution(C,V) :- puzzle(C,V).
"""
puzzle.solution_program += enc_library['ring_around_cell']
puzzle.solution_program += """
    % Count neighboring mines
    counts(C,R,P,S) :- cell(C),
        R = #count { D : cell(D), D != C, ring_around_cell(C,D,1), solution(D,mine(r)) },
        P = #count { D : cell(D), D != C, ring_around_cell(C,D,1), solution(D,mine(p)) },
        S = #count { D : cell(D), D != C, ring_around_cell(C,D,1), solution(D,mine(s)) }.
    
    % If there is a majority of one type of mine, that's the winner
    :- counts(C,R,P,S), R > P, R > S, not solution(C,mine(T)) : sign(T); not solution(C,r).
    :- counts(C,R,P,S), P > R, P > S, not solution(C,mine(T)) : sign(T); not solution(C,p).
    :- counts(C,R,P,S), S > R, S > P, not solution(C,mine(T)) : sign(T); not solution(C,s).
    
    % If there is a three-way tie, 0 is the winner
    :- counts(C,N,N,N), not solution(C,mine(T)) : sign(T); not solution(C,0).
    
    % If there is a two-way tie, use rock-paper-scissors rules
    :- counts(C,R,P,S), R = P, R > S, not solution(C,mine(T)) : sign(T); not solution(C,p).
    :- counts(C,R,P,S), R = S, R > P, not solution(C,mine(T)) : sign(T); not solution(C,r).
    :- counts(C,R,P,S), P = S, P > R, not solution(C,mine(T)) : sign(T); not solution(C,s).
"""
puzzle.essential_solution_constraints = [
    """
    % Equal number of mines of each type
    :- R = #count { D : cell(D), solution(D,mine(r)) },
        P = #count { D : cell(D), solution(D,mine(p)) },
        R != P.
    :- R = #count { D : cell(D), solution(D,mine(r)) },
        S = #count { D : cell(D), solution(D,mine(s)) },
        R != S.
    :- P = #count { D : cell(D), solution(D,mine(p)) },
        S = #count { D : cell(D), solution(D,mine(s)) },
        P != S.
    """,
]

In [3]:
puzzle.generate(
    verbose=True,
    precompute_solution=False,
)

Done grounding..
..
Stopped after solving timeout..
Solving time: 314.58 seconds



In [4]:
print(puzzle.pretty_repr_puzzle())

|   |   | s | p |   |
|   |   |   |   | r |
|   |   | p | p |   |
| r | r | r |   |   |
|   | r | r | r | r |


In [5]:
print(puzzle.pretty_repr_solution())

|[p]|[s]| s | p |[r]|
|[p]|[s]|[p]|[p]| r |
|[s]| s | p | p |[r]|
| r | r | r |[r]|[s]|
|[r]| r | r | r | r |
